<a href="https://colab.research.google.com/github/Tomawock/MLDM_COVID-19/blob/master/covid_test_elaboration_dataset_noRnd_doppio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import joblib
from datetime import datetime

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import r2_score, mean_squared_error, max_error
from sklearn.ensemble import RandomForestRegressor 

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [7]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/Shared drives/MLDM progetto/Datasets/dataset_risultati_test')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df = pd.read_csv('covid_extra_trees_test.csv')
df2 = pd.read_csv('covid_extra_trees_noRndDs_test.csv')
df3 = pd.read_csv('covid_extra_trees_doppio_test.csv')

In [20]:
v=[]
v.append(df.mean(axis = 0))
v.append(df2.mean(axis = 0))
v.append(df3.mean(axis = 0))
v.append(np.zeros(37))
# print(len(v))
# print(v[0])

In [21]:
df_results=pd.DataFrame(columns=df.columns, index=['RND_DAYS','NO_RND_DAYS', 'DATASET_DOPPIO', 'RMSE_DELTA'])
print(df_results.shape)

(4, 37)


In [22]:
df_results.loc['RND_DAYS'] = v[0]
df_results.loc['NO_RND_DAYS'] = v[1]
df_results.loc['DATASET_DOPPIO'] = v[2]
df_results.loc['RMSE_DELTA'] = v[3]

Risultati analizzando i dataset con il campione casuale di giorni all'interno dei vari intervalli temporali, o definendo a priori i giorni da analizzare

In [23]:
del df_results['Unnamed: 0']
df_results

,R2_fut1_nuovi_positivi,RMSE_fut1_nuovi_positivi,MAXERR_fut1_nuovi_positivi,R2_fut2_nuovi_positivi,RMSE_fut2_nuovi_positivi,MAXERR_fut2_nuovi_positivi,R2_fut1_ricoverati_con_sintomi,RMSE_fut1_ricoverati_con_sintomi,MAXERR_fut1_ricoverati_con_sintomi,R2_fut2_ricoverati_con_sintomi,RMSE_fut2_ricoverati_con_sintomi,MAXERR_fut2_ricoverati_con_sintomi,R2_fut1_terapia_intensiva,RMSE_fut1_terapia_intensiva,MAXERR_fut1_terapia_intensiva,R2_fut2_terapia_intensiva,RMSE_fut2_terapia_intensiva,MAXERR_fut2_terapia_intensiva,R2_fut1_deceduti,RMSE_fut1_deceduti,MAXERR_fut1_deceduti,R2_fut2_deceduti,RMSE_fut2_deceduti,MAXERR_fut2_deceduti,R2_fut1_totale_ospedalizzati,RMSE_fut1_totale_ospedalizzati,MAXERR_fut1_totale_ospedalizzati,R2_fut2_totale_ospedalizzati,RMSE_fut2_totale_ospedalizzati,MAXERR_fut2_totale_ospedalizzati,R2_fut1_dimessi_guariti,RMSE_fut1_dimessi_guariti,MAXERR_fut1_dimessi_guariti,R2_fut2_dimessi_guariti,RMSE_fut2_dimessi_guariti,MAXERR_fut2_dimessi_guariti
RND_DAYS,0.8665,224.156,734.238,0.814633,293.194,1273.72,0.981267,531.838,2145.71,0.979767,555.679,2477.51,0.9823,58.8278,249.106,0.982633,58.023,246.343,0.994533,351.067,1300.49,0.993433,375.49,1402.29,0.9804,601.281,2483.41,0.979967,612.144,2735.18,0.9971,1232.68,4162.42,0.996867,1317.58,4362.98
NO_RND_DAYS,0.847,238.888,796.434,0.809133,287.396,1162.32,0.982267,494.773,2159.08,0.978733,552.342,2664.69,0.9808,59.4383,288.613,0.9814,57.8948,282.49,0.997867,224.14,985.637,0.997133,246.431,1090.36,0.982067,563.181,2565.82,0.979733,599.413,2890.68,0.998867,825.893,2910.55,0.998733,845.664,3054.23
DATASET_DOPPIO,0.882,219.852,983.768,0.874667,229.859,1083.32,0.9958,256.893,1126.14,0.995067,277.075,1356.66,0.995333,30.9529,156.991,0.995733,29.9457,166.843,0.9988,170.301,744.134,0.9986,174.355,754.728,0.996,282.319,1241.95,0.995333,300.515,1521.16,0.999133,623.584,2658.93,0.999133,661.805,3413.61
RMSE_DELTA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
best_doppio_days=0
n_loops=0
percentage=[]
for i in df_results.columns:
  if 'RMSE_' in i:
    n_loops+=1
    if df_results.loc['RND_DAYS',i] > df_results.loc['NO_RND_DAYS',i] and df_results.loc['NO_RND_DAYS',i] > df_results.loc['DATASET_DOPPIO',i]:
      best_doppio_days+=1
      percentage.append(100-(df_results.loc['DATASET_DOPPIO',i]*100/min(df_results.loc['RND_DAYS',i], df_results.loc['NO_RND_DAYS',i])))
    

print(f"Numero di test nei quali è piu accurata la valutazione con il dataset aumentato: {best_doppio_days}/{n_loops}")
print(f"La differenza media in RMSE è di:\t{np.mean(percentage)}")


Numero di test nei quali è piu accurata la valutazione con il dataset aumentato: 10/12
La differenza media in RMSE è di:	36.54520092369354


In [27]:
best_row = []
for col in df3.columns:
  if 'RMSE' in col:
    best_row.append((col, df3.loc[df3[col].idxmin(), "Unnamed: 0"], df3[col].min()))

for row in best_row:
  print(row)



('RMSE_fut1_nuovi_positivi', 'dataset_finale_medie_doppio.csv:22', 152.796)
('RMSE_fut2_nuovi_positivi', 'dataset_finale_medie_doppio.csv:1', 177.15200000000002)
('RMSE_fut1_ricoverati_con_sintomi', 'dataset_finale_medie_doppio.csv:777', 166.673)
('RMSE_fut2_ricoverati_con_sintomi', 'dataset_finale_medie_doppio.csv:777', 183.447)
('RMSE_fut1_terapia_intensiva', 'dataset_finale_medie_doppio.csv:1', 21.01)
('RMSE_fut2_terapia_intensiva', 'dataset_finale_medie_doppio.csv:1', 22.534000000000002)
('RMSE_fut1_deceduti', 'dataset_finale_medie_2_week_doppio.csv:1', 140.222)
('RMSE_fut2_deceduti', 'dataset_finale_medie_1_week_doppio.csv:22', 139.918)
('RMSE_fut1_totale_ospedalizzati', 'dataset_finale_medie_doppio.csv:777', 198.65900000000002)
('RMSE_fut2_totale_ospedalizzati', 'dataset_finale_medie_doppio.csv:777', 211.987)
('RMSE_fut1_dimessi_guariti', 'dataset_finale_medie_1_week_doppio.csv:432145', 485.47)
('RMSE_fut2_dimessi_guariti', 'dataset_finale_medie_2_week_doppio.csv:1', 421.27099999